In [2]:
import numpy as np

wavelength_binned = np.concatenate([
    np.arange(15000, 17001, 0.3),
    np.arange(4713, 4902.05, 0.05),
    np.arange(7585, 7888.05, 0.05),
    np.arange(5645, 5873.05, 0.05)
])

In [3]:
print(len(wavelength_binned))

21076
